In [1]:
import json
import csv
import shapely

from pandas import DataFrame, read_csv

def load_counties_data():
    with open("./counties.geojson") as f:
        counties_geojson = json.load(f)
      
    co_data_frame = read_csv('./co-est2024-alldata.csv', encoding='iso-8859-1')

    for feature in counties_geojson["features"]:
        coords_stack = [ feature["geometry"]["coordinates"] ]

        while isinstance(coords_stack[-1][0], list):
            coords_stack.append(coords_stack[-1][0]) 

        coords = coords_stack[-2]    
        try: 
            raw_polygon = shapely.geometry.Polygon(coords) 
            state = feature["properties"]["STATEFP"]
            county = feature["properties"]["COUNTYFP"]
            co_data_frame.loc[co_data_frame.query(f"STATE == {state} and  COUNTY == {county}").index, "centroid"] = raw_polygon.centroid
        except Exception as e:
            print(e)
            print("Invalid coordinates:", feature["properties"]["Name"])  
    return (counties_geojson, co_data_frame)

(counties_geojson, co_data_frame) = load_counties_data()


In [2]:
import numpy as np
import pandas as pd
import json

def haversine_miles(centroid, lat2, lon2):
    r = 3958.8  # earth radius miles
    lat1 = centroid.y
    lon1 = centroid.x
    phi1 = np.radians(lat1); phi2 = np.radians(lat2)
    dphi = phi2 - phi1
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2
    return 2*r*np.arcsin(np.sqrt(a))

leagues = {
    'NFL': {
        "weight": 0.9
    },
    'NBA': {
        "weight": 0.6
    },
    'MLB': {
        "weight": 0.5
    },
    'NHL': {
        "weight": 0.25
    }, 
    'MLS': {
        "weight":0.15
    }
}    


for league in leagues.keys():
  with open(f"./teams_{league}.json") as f:
    leagues[league]["json"] = json.load(f)

    teams = pd.DataFrame(leagues[league]["json"]["teams"])

    teams['S'].fillna(0.0, inplace=True)
    teams['L'].fillna(0.5, inplace=True)
       
    # compute distance matrix
    d = np.zeros((len(co_data_frame), len(teams)))
    for i, c in co_data_frame.iterrows():
        if not pd.isna(c["centroid"]):
            for j, t in teams.iterrows():
                d[i,j] = haversine_miles(c["centroid"], t["coordinates"]['lat'], t["coordinates"]['lon'])


    leagues[league]["teams"] = teams   
    leagues[league]["distances"] = d  

/tmp/ipykernel_2273380/2195668802.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  teams['S'].fillna(0.0, inplace=True)
/tmp/ipykernel_2273380/2195668802.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [3]:
import numpy as np
import pandas as pd

distance_decay = 0.005  
competition_temperature = 0.05     
not_nearest_multiplier = 2.0 # added to distance multiplied (d - nearest_d) 
not_same_state_multiplier = 2.0 # TODO distance multipled if not same state

for league in leagues.keys(): 
    d = leagues[league]["distances"] 
    
    # compute raw R_{c,t}
    nearest_key = "nearest"
    R = np.zeros_like(d)
    league_weight = leagues[league]["weight"]

    co_data_frame[nearest_key] = float('nan')
    for j, t in leagues[league]["teams"].iterrows():
        for i, c in co_data_frame.iterrows():
            nearest = co_data_frame.loc[i, nearest_key]
            if pd.isna(nearest) or d[i,j] < nearest:
                co_data_frame.loc[i, nearest_key] = d[i,j] 
    
    for j, t in leagues[league]["teams"].iterrows():
        for i, c in co_data_frame.iterrows():
            effective_d = d[i,j]
            nearest_d = c[nearest_key]
            if not pd.isna(nearest_d) and effective_d > nearest_d:
                effective_d = nearest_d + ((effective_d - nearest_d) * not_nearest_multiplier)
            if c["STNAME"] != t["state"]:
                effective_d *= not_same_state_multiplier     
            D = np.exp(-distance_decay * effective_d * (1.0 - t["L"])) 
            R[i,j] = league_weight * D * (1 + t["S"]) * t["L"] 

    expR = np.exp(R / competition_temperature)
    leagues[league]["shares"] =  expR / expR.sum(axis=1, keepdims=True)    
    leagues[league]["Rs"] = R
  

In [4]:
for league in leagues.keys(): 
    d = leagues[league]["distances"]
    R = leagues[league]["Rs"]
    
    shares = leagues[league]["shares"]
    # fans = (shares * co_data_frame['POPESTIMATE2020'].values.reshape(-1,1)).astype(int)
    
    df_out = []
    for i, c in co_data_frame.iterrows():
        for j, t in leagues[league]["teams"].iterrows():
            if shares[i,j] > 0.01:
                df_out.append({
                    'county': c['CTYNAME'],
                    'countyfp': c['COUNTY'],
                    'state': c['STNAME'],
                    'statefp': c['STATE'],
                    'team_name': t['name'],
                    'population': c['POPESTIMATE2020'],
                    # 'distance_miles': round(d[i,j],1),
                    # 'raw_R': R[i,j],
                    'share': shares[i,j],
                    # 'fans': fans[i,j]
            })
    leagues[league]["dfs"] = pd.DataFrame(df_out)

In [5]:
default_style = {
        "color": "grey",
        "weight": 1,
        "fillColor": "grey",
        "fillOpacity": 0.1,
    }

def opactity_for_population(population): 
    if population > 5000000:
        return 0.9
    if population > 2000000:
        return 0.8
    if population > 1000000:
        return 0.7  
    if population > 500000:
        return 0.6 
    if population > 100000:
        return 0.5 
    if population > 50000:
        return 0.4
    if population > 10000:
        return 0.3
    if population > 5000:
        return 0.2
    return 0.1

for feature in counties_geojson["features"]:
    feature["properties"]["style"] = default_style
                

def highlight(league, team_name, color, share_theshold = 0.05): 
    for feature in counties_geojson["features"]:
       state = feature["properties"]["STATEFP"]
       county = feature["properties"]["COUNTYFP"]
       county_rows = leagues[league]["dfs"].query(f"statefp == {state} & countyfp == {county}").sort_values(by="share", ascending=False)
       if county_rows.shape[0] > 0:
           county_row = county_rows.iloc[0]
           if (not team_name or county_row["team_name"] == team_name) and county_row['share'] > share_theshold:                
                feature["properties"]["style"] = {
                "color": "grey",
                "weight": 1,
                "fillColor": color,
                "fillOpacity": opactity_for_population(county_row["population"] * county_row["share"])
                }               
           elif not team_name:
               feature["properties"]["style"] = {
                "color": "grey",
                "weight": 1,
                "fillColor": "green",
                "fillOpacity": opactity_for_population(county_row["population"])
               }
   
            
            
highlight("MLB", None, "blue")

highlight("MLB", "St. Louis Cardinals", "red")

#highlight("NHL", "St. Louis Blues", "blue")
#highlight("NHL", "Chicago Blackhawks", "black")
#highlight("NHL", "Minnesota Wild", "green")
#highlight("NHL", "Nashville Predators", "yellow")
#highlight("NHL", "Detroit Red Wings", "red")

In [6]:
from ipyleaflet import Map, Marker, MarkerCluster, GeoJSON, Popup, FullScreenControl
from shapely.geometry import Polygon, Point
from shapely.ops import transform
import ipywidgets as widgets
import pandas as pd

map = Map(center=[38.533899, -90.50261], zoom=8, scroll_wheel_zoom=True)

def show_teams(event, feature, **kwargs): 
    statefp = feature["properties"]["STATEFP"]
    countyfp = feature["properties"]["COUNTYFP"]
    row = co_data_frame.query(f"STATE == {statefp} & COUNTY == {countyfp}") 
    centroid = row["centroid"].iloc[0]
    population = row["POPESTIMATE2020"].iloc[0]

    all_county_rows = pd.DataFrame()    
    leagues_table = ""  
    for league in leagues.keys():
        county_rows = leagues[league]["dfs"].query(f"statefp == {statefp} & countyfp == {countyfp}") 
        county_rows["league"] = league
        all_county_rows = pd.concat([all_county_rows, county_rows], ignore_index=True)

    all_county_rows = all_county_rows.sort_values(by="share", ascending=False)
    for i, county_row in all_county_rows.iterrows():
        if county_row['share'] > 0.05:
            leagues_table += (
             "<tr>"  
             f"<td>{county_row['league']}</td>" 
             f"<td>{county_row['team_name']}</td>" 
             f"<td>{round(county_row['share'] * 100, 1)}%</td>" 
             # f"<td>{county_row['fans']}</td>" 
             "</tr>")
       
    popup = Popup(location=(centroid.y, centroid.x), 
                  child=widgets.HTML("<table border='1' style='border-collapse: collapse'>" +
                  f'<caption>{feature["properties"]["Name"]} - {population}</caption>' +    
                  leagues_table +
                  "</table>"  
        )
    )
    map.add(popup)    
    

layer = GeoJSON(data = counties_geojson, 
    hover_style = {"fillColor": "white"}
)

layer.on_click(show_teams)

map.add(layer)

map.add(FullScreenControl())


Map(center=[38.533899, -90.50261], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [7]:
map.fullscreen = True